In [67]:
# guide : https://riot-api-libraries.readthedocs.io/en/latest/collectingdata.html 

import os
import requests
import pandas as pd
from cassiopeia import Champion, Champions
import pickle
import time

In [92]:
## Get account details by providing the account name
def requestSummonerData(region,summonerName, APIKey):
    URL = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}?api_key={APIKey}"
    response = requests.get(URL)
    return response.json()

def requestSummonerPerformance(region,summoner_id, APIKey):
    URL = "https://{}.api.riotgames.com/lol/league/v4/entries/by-summoner/{}?api_key={}".format(region,summoner_id,APIKey)
    response = requests.get(URL)
    return response.json()

def requestMatchList(region,acctID, APIKey):
    URL = f"https://{region}.api.riotgames.com/lol/match/v4/matchlists/by-account/{acctID}?api_key={APIKey}"
    response = requests.get(URL)
    return response.json()

def requestMatchInfo(region,matchID, APIKey):
    URL = "https://{}.api.riotgames.com/lol/match/v4/matches/{}?api_key={}".format(region,matchID,APIKey)
    response = requests.get(URL)
    return response.json()

def requestLeagueInfo(region,leagueID, APIKey):
    URL = "https://{}.api.riotgames.com/lol/league/v4/leagues/{}?api_key={}".format(region,leagueID,APIKey)
    response = requests.get(URL)
    return response.json()


# load champion names and IDs
def define_champ_names():
    df_champs = pd.DataFrame(columns=['champion_name','champion_ID'])
    champions = Champions(region="NA")
    index = 0
    for champion in champions:
            df_champs.loc[index] = [champion.name, champion.id ]
            index+=1
    return df_champs


def find_player_idx(match_info, summoner_name, acct_ID):
    # find index of player in player list
    for idx, participant in enumerate(match_info['participantIdentities']):
        if participant['player']['accountId'] == acct_ID and participant['player']['summonerName'] == summoner_name:
            player_key = idx
            
    return player_key

def calc_kda(statsDict): 
    if statsDict['deaths'] == 0:
        kda = statsDict['kills'] + statsDict['assists']
    else:
        kda = ( statsDict['kills'] + statsDict['assists'] ) / statsDict['deaths']
    return kda


def get_highest_rank(matchInfo, playerKey):
    if 'highestAchievedSeasonTier' not in matchInfo['participants'][playerKey]:
        thisRank = 'Unranked'
        matchRank = 0
    else:
        thisRank = matchInfo['participants'][playerKey]['highestAchievedSeasonTier']
        matchRank = rankNames.index(thisRank) + 1
    return matchRank

In [93]:
def main_grab_data(region,summoner_name, APIKey):
    
    print(summoner_name)
    
    # initialize info
    df_user_matches = pd.DataFrame()
    dfChampNames = define_champ_names()
    
    # grab summoner data
    summoner_dat = requestSummonerData(region,summoner_name, APIKey)
    if 'accountId' in summoner_dat:
        acctID = summoner_dat['accountId']
    else:
        raise Exception('Invalid API key')
    
    # grab summoner general performance stats
    summoner_performance = requestSummonerPerformance(region,summoner_dat['id'], APIKey)
    player_rank = summoner_performance[0]['tier']

    # grab summoner match list
    matchList  = requestMatchList(region,acctID, APIKey)   
    numMatches = len(matchList ['matches'])

    # go through each match
    for iMatch in range(numMatches):

        # need to pause bc of rate limits for riotAPI
        if iMatch%40 == 0 and iMatch != 0: 

            time.sleep(121)

        matchID = matchList['matches'][iMatch]['gameId'] # get this match's ID

        # get request match data and extract metrics from match info
        match_info = requestMatchInfo(region,matchID, APIKey) # pull this game's info from riotAPI
        max_time = match_info['gameDuration']
        game_version = match_info['gameVersion']
        player_key = find_player_idx(match_info, summoner_name, acctID)
        stats_dict = match_info['participants'][player_key]['stats'] # get stats dict from this game
        playerTeam = match_info['participants'][player_key]['teamId']

        ############ preprocess data

        # calculate KDA
        kda = calc_kda(stats_dict)

        # figure out champion name from ID
        thisChampionID = match_info['participants'][player_key]['championId']
        tfIndex = dfChampNames['champion_ID'] == thisChampionID
        champName =  dfChampNames[tfIndex]['champion_name'].item()

        # figure out player's match rank
        #match_rank = get_highest_rank(match_info, player_key)

        # get role and lane
        role = match_info['participants'][player_key]['timeline']['role']
        lane = match_info['participants'][player_key]['timeline']['lane']
        teamID = match_info['participants'][player_key]['teamId'] 

        all_player_dict = {}
        for iParticipant in range(0,10):
            this_role = match_info['participants'][iParticipant]['timeline']['role']
            this_lane = match_info['participants'][iParticipant]['timeline']['lane']
            this_teamID = match_info['participants'][iParticipant]['teamId']

            thisPlayerChampionID = match_info['participants'][iParticipant]['championId']
            champIndex = dfChampNames['champion_ID'] == thisPlayerChampionID
            thisChampName =  dfChampNames[champIndex]['champion_name'].item()

            tmpID = f"{this_role}_{this_lane}"

            if this_lane == 'TOP' and this_teamID == teamID: 
                all_player_dict['player_top'] = thisChampName
            elif this_lane == 'JUNGLE' and this_teamID == teamID:
                all_player_dict['player_jung'] = thisChampName
            elif this_lane == 'MIDDLE' and this_teamID == teamID:
                all_player_dict['player_mid'] = thisChampName
            elif this_role == 'DUO_CARRY' and this_teamID == teamID:
                all_player_dict['player_ADC'] = thisChampName
            elif this_role == 'DUO_SUPPORT' and this_teamID == teamID:
                all_player_dict['player_supp'] = thisChampName
            elif this_lane == 'TOP' and this_teamID != teamID:
                all_player_dict['opp_top'] = thisChampName
            elif this_lane == 'JUNGLE' and this_teamID != teamID:
                all_player_dict['opp_jung'] = thisChampName
            elif this_lane == 'MIDDLE' and this_teamID != teamID:
                all_player_dict['opp_mid'] = thisChampName
            elif this_role == 'DUO_CARRY' and this_teamID != teamID:   
                all_player_dict['opp_ADC'] = thisChampName
            elif this_role == 'DUO_SUPPORT' and this_teamID != teamID: 
                all_player_dict['opp_supp'] = thisChampName

        if len(all_player_dict) != 10:
            continue

        ############ put data together 

        if role in ['SOLO','NONE']:
            true_role = lane
        else:
            true_role = role


        ######### add data to dataframes to save
        if df_user_matches.shape[0] == 0:
            next_idx = 0
        else:
            next_idx = df_user_matches.index[-1]+1

        df_user_matches.loc[next_idx, 'acct_id'] = acctID
        df_user_matches.loc[next_idx, 'match_id'] = matchID
        df_user_matches.loc[next_idx, 'match_rank'] = player_rank
        df_user_matches.loc[next_idx, 'role'] = role
        df_user_matches.loc[next_idx, 'champ'] = thisChampName
        df_user_matches.loc[next_idx, 'win'] = int(stats_dict['win'])

        df_user_matches.loc[next_idx, 'kills'] = stats_dict['kills']
        df_user_matches.loc[next_idx, 'deaths'] = stats_dict['deaths']
        df_user_matches.loc[next_idx, 'assists'] = stats_dict['assists']
        df_user_matches.loc[next_idx, 'gold_earned'] = stats_dict['goldEarned']
        df_user_matches.loc[next_idx, 'acct_id'] = acctID

        df_user_matches.loc[next_idx, 'player_top'] = all_player_dict['player_top']
        df_user_matches.loc[next_idx, 'player_jung'] = all_player_dict['player_jung']
        df_user_matches.loc[next_idx, 'player_mid'] = all_player_dict['player_mid']
        df_user_matches.loc[next_idx, 'player_ADC'] = all_player_dict['player_ADC']
        df_user_matches.loc[next_idx, 'player_supp'] = all_player_dict['player_supp'],

        df_user_matches.loc[next_idx, 'opp_top'] = all_player_dict['opp_top']
        df_user_matches.loc[next_idx, 'opp_jung'] = all_player_dict['opp_jung']
        df_user_matches.loc[next_idx, 'opp_mid'] = all_player_dict['opp_mid']
        df_user_matches.loc[next_idx, 'opp_ADC'] = all_player_dict['opp_ADC'] 
        df_user_matches.loc[next_idx, 'opp_supp'] = all_player_dict['opp_supp'] 

        print('Added match #' + str(iMatch))

    return df_user_matches


In [94]:
if __name__ == "__main__":
    
    ## Parameters
    APIKey = os.environ.get('League_API')
    region = 'na1'
    summoner_name = 'Duvet Cover'
    flag_save_load = 0 # 1 if save, 2 if load
    
    df_user_matches = main_grab_data(region,summoner_name, APIKey)
    
    if flag_save_load == 1:
        with open(r'.\sample_match_info.pkl', 'wb') as handle:
            pickle.dump(match_info, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    elif flag_save_load == 2:
        with open(r'.\sample_match_info.pkl', 'rb') as handle:
            match_info = pickle.load(handle)

Duvet Cover
Added match #0
Added match #2
Added match #6
Added match #8
Added match #9
Added match #10
Added match #11
Added match #12
Added match #13
Added match #14
Added match #15
Added match #19
Added match #23
Added match #26
Added match #27
Added match #29
Added match #30
Added match #34
Added match #35
Added match #37
Added match #38
Added match #39
Added match #40
Added match #42
Added match #44
Added match #46
Added match #48
Added match #50
Added match #52
Added match #53
Added match #54
Added match #58
Added match #60
Added match #61
Added match #65
Added match #66
Added match #67
Added match #68
Added match #69
Added match #70
Added match #72
Added match #74
Added match #76
Added match #77
Added match #78
Added match #80
Added match #81
Added match #82
Added match #83
Added match #86
Added match #88
Added match #89
Added match #92
Added match #93
Added match #94
Added match #96
Added match #99


In [95]:
df_user_matches.head()

,acct_id,match_id,match_rank,role,champ,win,kills,deaths,assists,gold_earned,player_top,player_jung,player_mid,player_ADC,player_supp,opp_top,opp_jung,opp_mid,opp_ADC,opp_supp
0,NJmzpDSPvy2esijBBelXuRVm9JZu3-tYKOi71qOOfYqgh5Y,3.483420e+09,GOLD,DUO_SUPPORT,Caitlyn,0.0,2.0,1.0,8.0,7483.0,Garen,Zac,Ahri,Varus,Sona,Lucian,Kayn,Syndra,Caitlyn,Leona
1,NJmzpDSPvy2esijBBelXuRVm9JZu3-tYKOi71qOOfYqgh5Y,3.481637e+09,GOLD,DUO_SUPPORT,Nunu & Willump,1.0,1.0,1.0,19.0,6178.0,Nunu & Willump,Rammus,Qiyana,Twitch,Sona,Singed,Kayn,Vladimir,Jinx,Nautilus
2,NJmzpDSPvy2esijBBelXuRVm9JZu3-tYKOi71qOOfYqgh5Y,3.416268e+09,GOLD,SOLO,Kled,1.0,8.0,5.0,3.0,9158.0,Garen,Fiddlesticks,Orianna,Yasuo,Ornn,Kled,Jax,Pantheon,Ezreal,Pyke
3,NJmzpDSPvy2esijBBelXuRVm9JZu3-tYKOi71qOOfYqgh5Y,3.416196e+09,GOLD,SOLO,Cho'Gath,1.0,10.0,2.0,0.0,9908.0,Gnar,Lee Sin,Orianna,Kai'Sa,Nami,Cho'Gath,Graves,LeBlanc,Kalista,Taric
4,NJmzpDSPvy2esijBBelXuRVm9JZu3-tYKOi71qOOfYqgh5Y,3.416109e+09,GOLD,SOLO,Camille,0.0,4.0,12.0,2.0,9609.0,Yasuo,Lee Sin,Nasus,Tristana,Thresh,Camille,Udyr,Zed,Ezreal,Zyra
